<a href="https://colab.research.google.com/github/nrohrbach/Infotracer/blob/main/Infotracer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infotracer Test
s

In [ ]:
import folium
from folium.plugins import Draw
from folium.plugins import Geocoder
from folium.utilities import JsCode
import requests
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Create a map centered on Switzerland
map = folium.Map(location=[46.8182, 8.2275],
                 zoom_start=8,
                 height="600px",
                 width="1000px",
                 control_scale=True,
                 tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
                 attr='Map data: &copy; <a href="https://www.swisstopo.ch" target="_blank" rel="noopener noreferrer">swisstopo</a>'
                 )

# Feature Group wo eingzeichnete Geometrien gespeichert werden
folium.FeatureGroup(name="Geometrien").add_to(map)

# JavaScript code to store coordinates in variables
js_code = """
function(e) {
    var lat = e.latlng.lat;
    var lng = e.latlng.lng;

    // Store the coordinates in JavaScript variables
    var myLat = lat;
    var myLng = lng;

    // Log the coordinates to the console for verification
    console.log("Latitude:", myLat);
    console.log("Longitude:", myLng);

    // Optionally, you can send the coordinates to Python using a custom function
    // Example:  google.colab.kernel.invokeFunction('store_coordinates', [myLat, myLng], {});
}
"""

# Zeichenfunktion einführen.
Draw(
    export=False,
    show_geometry_on_click=True,
    position='topleft',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'circlemarker': False,
        'marker': True,
        'rectangle': False,
    },
    edit_options={'edit': False,
                  'remove':False},
    on={
        "draw:created": JsCode(js_code) # Use draw:created event
    },
).add_to(map)

Geocoder(
    add_marker=False,
    zoom=15
).add_to(map)


map

## Todo
Koordinaten abfangen und weiterverwenden

In [ ]:
# Temporär Koordinaten und Radius eingeben
#####################
coordinates= [9.843278,46.497595]
Radius = 5000
#####################

easting = coordinates[0]
northing = coordinates[1]

## Transformation der Koordinaten von WGS84 in LV95
Die swisstopo API mit Distanzangabe läuft am Besten in LV95. Grund ist dass LV95 bereits in Meter ist. WGS84 in Grad.

In [ ]:
def get_lv95_coordinates(easting, northing):
    url = f"https://geodesy.geo.admin.ch/reframe/wgs84tolv95?easting={easting}&northing={northing}&format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

lv95_data = get_lv95_coordinates(easting, northing)

if lv95_data:
    #print(json.dumps(lv95_data, indent=2)) #Nicely formatted JSON output
    # Access specific coordinate values
     easting_lv95 = float(lv95_data["easting"])
     northing_lv95 = float(lv95_data["northing"])

# Abfrage des APIs

In [ ]:
#Bounding Box für Radius Abfrage erstellen
mapExtent = [0,0,0,0]
mapExtent[0] = easting_lv95-Radius
mapExtent[1] = northing_lv95-Radius
mapExtent[2] = easting_lv95+Radius
mapExtent[3] = northing_lv95+Radius

In [ ]:
def query_geo_admin_api(coordinates):
    """Queries the Swiss Geo Admin API for information at given coordinates."""

    url = "https://api3.geo.admin.ch/rest/services/all/MapServer/identify"
    params = {
        "geometry":  f"{mapExtent[0]},{mapExtent[1]},{ mapExtent[2]},{mapExtent[3]}",  # Longitude, Latitude
        "geometryFormat": "geojson",
        "geometryType": "esriGeometryEnvelope",
        "sr": "2056",
        "lang": "de",
        "layers": "all:ch.bafu.hydrogeologie-markierversuche",
        "returnGeometry": "true",
        "tolerance": 0
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error querying Geo Admin API: {e}")
        return None

api_response = query_geo_admin_api(coordinates)



# Antworten der API als Dataframe speichern und auf Karte darstellen

In [ ]:
# Antworten des API als Dataframe speichern
if api_response:
    results = []
    for feature in api_response['results']:
        result = {
            'x': feature["properties"]['x'],
            'y': feature["properties"]['y'],
            'ort': feature["properties"]['ort'],
            'datum': feature["properties"]['datum'],
            'milieu': feature["properties"]['milieu'],
            'marker': feature["properties"]['markierstoff'],
            'menge': feature["properties"]['menge_einheit'],
            'label' : feature["properties"]['label']
        }
        results.append(result)
    else:
        print("No results found.")
# Create a Pandas DataFrame
df = pd.DataFrame(results)

# Create a Geodataframe
dfgeo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y),crs='EPSG:2056')

# Startpunkt als Geodataframe speichern als Grundlage für Distanzberechnung
startpunkt = Point(easting_lv95, northing_lv95)
startpunkt_geoseries = gpd.GeoSeries([startpunkt], crs=dfgeo.crs)

#Distanz von Resultaten zu Startpunkt berechnen
dfgeo['Distanz'] = round(dfgeo.distance(startpunkt_geoseries[0]),0)

# Convert the GeoDataFrame's CRS to WGS84 (latitude/longitude) for Folium
dfgeo = dfgeo.to_crs(epsg=4326)

# Create a map centered on Switzerland
mapresults = folium.Map(location=[northing,easting],
                 zoom_start=14,
                 height="600px",
                 width="1000px",
                 control_scale=True,
                 tiles="https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{z}/{x}/{y}.jpeg",
                 attr='Map data: &copy; <a href="https://www.swisstopo.ch" target="_blank" rel="noopener noreferrer">swisstopo</a>'
                 )
if api_response:
    # Add markers to the map
  for index, row in dfgeo.iterrows():
      folium.Marker(
          location=[row.geometry.y, row.geometry.x],  # Latitude, Longitude
          popup=f"Ort: {row['ort']}<br>Datum: {row['datum']}<br>Milieu: {row['milieu']}<br>Marker: {row['marker']}<br>Menge: {row['menge']},<br>Distanz (m): {row['Distanz']}",
          tooltip=row['label'],
      ).add_to(mapresults)



# Add Startpunkt to the map
folium.Marker(
    location=  [northing,easting],  # Latitude, Longitude
    popup=f"Startpunkt",
    tooltip=f"Startpunkt",
    icon=folium.Icon(color='green')
).add_to(mapresults)


mapresults






No results found.


In [ ]:
dfgeo

,x,y,ort,datum,milieu,marker,menge,label,geometry,Distanz
0,2521145.0,1155950.0,Apples,28.05.1985,None,Fluorescéine,2 kg,136,POINT (6.41044 46.55017),2023.0
1,2519360.0,1156040.0,Ballens,10.09.1985,None,Fluorescéine,1 kg,220,POINT (6.38715 46.55077),341.0
2,2519520.0,1156580.0,Ballens,10.09.1985,None,Fluorescéine,1 kg,221,POINT (6.38914 46.55565),453.0
3,2519620.0,1156500.0,Ballens,10.09.1985,None,Iodid,5 kg,222,POINT (6.39046 46.55494),502.0
4,2521145.0,1155950.0,Apples,27.10.1986,None,Fluorescéine,2 kg,434,POINT (6.41044 46.55017),2023.0
5,2519600.0,1154600.0,Yens,21.09.1988,None,Duasyn,1 kg,967,POINT (6.39053 46.53785),1769.0
6,2519850.0,1154650.0,Yens,21.09.1988,None,Eosin,1 kg,968,POINT (6.39378 46.53833),1802.0
7,2519850.0,1154800.0,Yens,21.09.1988,None,Fluorescéine,1 kg,969,POINT (6.39375 46.53968),1664.0
8,2520775.0,1154500.0,Yens,21.09.1988,None,NaCl,1000 kg,971,POINT (6.40586 46.53709),2431.0
9,2521080.0,1155580.0,Apples,24.08.1992,None,Duasyn,1.5 kg,2504,POINT (6.40965 46.54684),2060.0
